In [1]:
import pandas as pd
# Importing and cleaning the data
data = pd.read_csv('lab_courses_2015.csv', sep=';')
data.head()

,id,component,action,target,objectid,edulevel,contextid,contextlevel,contextinstanceid,userid,courseid,anonymous,timecreated,cluster
0,3,mod_book,viewed,chapter,31566.0,2,105734,70,67525,14744,1089,0,1420079066,0
1,11,mod_book,viewed,course_module,2294.0,2,66321,70,41549,14731,630,0,1420100897,0
2,12,mod_book,viewed,chapter,13546.0,2,66321,70,41549,14731,630,0,1420100897,0
3,13,mod_book,viewed,chapter,13547.0,2,66321,70,41549,14731,630,0,1420100918,0
4,16,mod_book,viewed,course_module,2293.0,2,66331,70,41540,14731,630,0,1420102170,0


In [2]:
# replacing values for component
data['component'].replace(['assignsubmission_comments', 
                           'assignsubmission_file', 
                           'assignsubmission_onlinetext', 
                           'booktool_print', 
                           'core', 
                           'gradereport_overview', 
                           'gradereport_user', 
                           'mod_assign', 
                           'mod_book',
                           'mod_data',
                           'mod_feedback',
                           'mod_folder',
                           'mod_forum',
                           'mod_glossary', 
                           'mod_choice', 
                           'mod_page', 
                           'mod_quiz', 
                           'mod_resource', 
                           'mod_url'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], inplace=True)

In [3]:
# replacing values for action
data['action'].replace(['abandoned', 
                        'created', 
                        'deleted', 
                        'printed', 
                        'core', 
                        'searched', 
                        'started', 
                        'submitted', 
                        'updated',
                        'uploaded',
                        'viewed',
                           ],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)

In [4]:
# replacing values for target
data['target'].replace(['assessable', 
                        'attempt', 
                        'book', 
                        'comment', 
                        'course', 
                        'course_module', 
                        'course_module_completion', 
                        'discussion', 
                        'discussion_subscription',
                        'entry',
                        'grade_report',
                        'chapter', 
                        'post', 
                        'response', 
                        'submission', 
                        'subscription'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], inplace=True)

In [5]:
# check version number
import imblearn
print(imblearn.__version__)

0.10.1


In [6]:
from collections import Counter
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder

# split into input and output elements
X = data[['component', 'action', 'target', 'contextid', 'contextlevel', 'contextinstanceid', 'timecreated']]
y = data[['cluster']]

# label encode the target variable
y = LabelEncoder().fit_transform(y)

# summarize distribution
counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100

C:\Users\jpecu\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
from imblearn.under_sampling import NearMiss

# define the undersampling method
undersample = NearMiss(version=2, n_neighbors=3)

# transform the dataset
X, y = undersample.fit_resample(X, y)

# summarize distribution
counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30)
model_knn.fit(X_train, y_train)
y_pred = model_knn.predict(X_test)

In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       675
           1       1.00      1.00      1.00       674
           2       0.92      0.96      0.94       675
           3       0.99      0.93      0.96       675

    accuracy                           0.96      2699
   macro avg       0.97      0.96      0.96      2699
weighted avg       0.97      0.96      0.96      2699



In [11]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.964801778436458
